In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
IMG_SIZE = 32


#Path
DATA_PATH= "/home/nishant/Desktop/Dataset/DevanagariHandwrittenCharacterDataset"
num_classes = len(os.listdir(DATA_PATH + "/Train"))

/home/nishant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Test Directory

TEST_DIR=DATA_PATH +"/Test"

In [3]:
def getCV2(image, IMG_SIZE):
    img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    return img

# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DATA_PATH+"/Train"):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

# Function to save all the images in numpy array
def save_data_to_array(IMG_SIZE, path=DATA_PATH+"/Train"):
    labels, _, _ = get_labels(path)

    for label in labels:
        cv2_vectors = []
        images = [path + "/" + label + '/' + image for image in os.listdir(path + '/' + label)]
        for image in tqdm(images, "Saving vectors of label - '{}'".format(label)):
            cv2V = getCV2(image, IMG_SIZE)
            cv2_vectors.append(cv2V)
        np.save(DATA_PATH +"/" + label + '.npy', cv2_vectors)

# Function for getting train and validation data

def get_train_test(split_ratio=0.7, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH + "/Train")

    # Getting first arrays
    X = np.load(DATA_PATH + "/" + labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(DATA_PATH + "/" + label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [4]:
# save all the images in numpy array

save_data_to_array(IMG_SIZE,DATA_PATH + "/Train")

Saving vectors of label - 'character_17_tha': 100%|██████████| 1700/1700 [00:00<00:00, 3588.15it/s]
Saving vectors of label - 'character_11_taamatar': 100%|██████████| 1700/1700 [00:00<00:00, 4767.37it/s]
Saving vectors of label - 'character_15_adna': 100%|██████████| 1700/1700 [00:00<00:00, 3817.68it/s]
Saving vectors of label - 'character_32_patalosaw': 100%|██████████| 1700/1700 [00:00<00:00, 4528.31it/s]
Saving vectors of label - 'character_19_dha': 100%|██████████| 1700/1700 [00:00<00:00, 4411.86it/s]
Saving vectors of label - 'character_31_petchiryakha': 100%|██████████| 1700/1700 [00:00<00:00, 4692.14it/s]
Saving vectors of label - 'character_21_pa': 100%|██████████| 1700/1700 [00:00<00:00, 4694.96it/s]


In [5]:
# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

#X = X_train.reshape(X_train.shape[0], IMG_SIZE, IMG_SIZE, 1)
#validate_x = X_test.reshape(X_test.shape[0], IMG_SIZE, IMG_SIZE, 1)


#Y = to_categorical(y_train)
#validate_y = to_categorical(y_test)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_validate = le.transform(y_test)
y_train = to_categorical(y_train, num_classes)
y_validate = to_categorical(y_test, num_classes)

im_shape = (IMG_SIZE, IMG_SIZE, 1)
X_train = X_train.reshape(X_train.shape[0], *im_shape)
X_test = X_test.reshape(X_test.shape[0], *im_shape)

In [7]:
import keras
import matplotlib.pyplot as plt
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, MaxPool2D
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.layers import SimpleRNN
from keras import initializers
from keras.optimizers import RMSprop
from keras.models import load_model
from keras import regularizers

In [8]:
# My cnn model

cnn = Sequential()
kernelSize = (3, 3)
ip_activation = 'relu'
cnn.add(Conv2D(filters=32, kernel_size=kernelSize, input_shape=im_shape, activation='relu'))
cnn.add(Conv2D(filters=64, kernel_size=kernelSize, activation=ip_activation))
cnn.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="same"))
cnn.add(Conv2D(filters=64, kernel_size=kernelSize, activation=ip_activation))
cnn.add(Conv2D(filters=64, kernel_size=kernelSize, activation=ip_activation))
cnn.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="same"))
cnn.add(Dropout(0.2))
cnn.add(Flatten())
cnn.add(Dense(units=128, activation=ip_activation, kernel_initializer='uniform'))
cnn.add(Dense(units=64, activation=ip_activation, kernel_initializer='uniform'))
op_activation = 'softmax'
cnn.add(Dense(units=num_classes, activation=op_activation, kernel_initializer='uniform'))
optimize = 'adam'
loss = 'categorical_crossentropy'
metrics = ['accuracy']
cnn.compile(optimizer=optimize, loss=loss, metrics=metrics)

In [12]:
# In the 20 epochs, the training accuracy is 98.99% and validation accuracy is 98.02 on the training dataset

history = cnn.fit(X_train, y_train,
                  batch_size=32, epochs=20,
                  validation_data=(X_test, y_validate))

Train on 54739 samples, validate on 23461 samples
Epoch 1/20
54739/54739 [==============================] - 86s 2ms/step - loss: 0.0440 - acc: 0.9880 - val_loss: 0.1020 - val_acc: 0.9809
Epoch 2/20
54739/54739 [==============================] - 90s 2ms/step - loss: 0.0458 - acc: 0.9882 - val_loss: 0.1069 - val_acc: 0.9800
Epoch 3/20
54739/54739 [==============================] - 93s 2ms/step - loss: 0.0439 - acc: 0.9886 - val_loss: 0.1095 - val_acc: 0.9802
Epoch 4/20
54739/54739 [==============================] - 94s 2ms/step - loss: 0.0438 - acc: 0.9883 - val_loss: 0.1113 - val_acc: 0.9785
Epoch 5/20
54739/54739 [==============================] - 94s 2ms/step - loss: 0.0426 - acc: 0.9891 - val_loss: 0.1472 - val_acc: 0.9686
Epoch 6/20
54739/54739 [==============================] - 96s 2ms/step - loss: 0.0519 - acc: 0.9880 - val_loss: 0.1252 - val_acc: 0.9762
Epoch 7/20
54739/54739 [==============================] - 96s 2ms/step - loss: 0.0434 - acc: 0.9892 - val_loss: 0.1000 - val_acc

In [13]:
# Function to predit label of the image using model

def predict(filename, model):
    path = os.path.join(TEST_DIR, filename)
    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    img = np.array(img)
    data = img.reshape(-1, IMG_SIZE,IMG_SIZE,1)
    res = get_labels()[0][np.argmax(model.predict(data))]
    return res

In [14]:
# Finding Testing Accuracy on the Test data which is given in the Dataset

ctr_true = 0
ctr_total = 0
labels = os.listdir(TEST_DIR)
# labels
for label in labels:
    images = os.listdir(TEST_DIR + "/" + label)
#     print(images)
    for img in images:
        out = predict(label + "/" + img, cnn)
        if out == label:
            ctr_true += 1
        ctr_total += 1
    
print(ctr_true/ctr_total)

# Testing Accuracy: 98.12%

0.981231884057971
